Yosep Kim <br>
CS 5661 Homework1 <br>
cin: 220612403

In [20]:

import numpy as np
import pandas as pd

file = '/home/cowboyuniverse/Workspace/_2018_Spring_CS5661/HW1/Cancer_small.csv'

cancer_data_df = pd.read_csv(file)


<b>a- Read the dataset file “Cancer.csv” (you should download it from CSNS), and store it in a
Pandas DataFrame. Check out the dataset. As you see, the dataset includes 9 numerical
features. The last column is the binary label (“1” means it is a malignant cancer, “0” means
it is a benign tumor). You will use all 9 features in this homework.</b>

In [21]:
cancer_data_df.tail()

,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Malignant_Cancer
145,3,1,1,1,2,1,2,1,1,0
146,9,7,7,5,5,10,7,8,3,1
147,10,8,8,4,10,10,8,1,1,1
148,1,1,1,1,2,1,3,1,1,0
149,5,1,1,1,2,1,3,1,1,0


In [22]:
cols = cancer_data_df.columns 
features = cols[:len(cols)-1] 
y_columns = cols[len(cols)-1]

X = cancer_data_df[features]  
y = cancer_data_df[y_columns]


<b>b- Use sklearn functions (see class tutorials for details) to split the dataset into testing and
training sets with the following parameters: test_size=0.3, random_state=2.
</b>

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

#split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

print(X.shape)
print(y.shape)

(150, 9)
(150,)


<b>
c- Use “Decision Tree Classifier” to predict Cancer based on the training/testing datasets
that you built in part (b). Then, calculate and report the accuracy of your classifier. Use
this command to define your tree:
my_DecisionTree = DecisionTreeClassifier(random_state=2).</b>

In [24]:
#  DecisionTreeClassifier is name of a "sklearn class" to perform "Decision Tree Classification" 

from sklearn.tree import DecisionTreeClassifier
my_DecisionTree = DecisionTreeClassifier(random_state=2)
my_DecisionTree.fit(X_train, y_train)
decisiontree_y_predict = my_DecisionTree.predict(X_test)

In [25]:

decisiontree_accuracy = accuracy_score(y_test, decisiontree_y_predict)
print('Decisiontree Accuracy:', decisiontree_accuracy)
my_DecisionTree_list = cross_val_score(my_DecisionTree, X, y, cv=10, scoring='accuracy')

my_DecisionTree_cv = my_DecisionTree_list.mean()
print('DecisionTree accuracy with cross validation:',my_DecisionTree_cv)


Decisiontree Accuracy: 0.866666666667
DecisionTree accuracy with cross validation: 0.894702380952


<b>d- Now, we want to perform “Bagging” based on 19 “base decision tree classifiers”. Note: you should write your own code to perform Bagging (don’t use scikit-learn functions for Bagging!) To do so, you need to perform bootstrapping first. You can write a “for” loop with loop variable i=0…18. In each iteration of the loop, you have to:

make a bootstarp sample of the original “Training” Dataset (build in part(b)) with size of bootstarp_size = 0.8*(Size of the original dataset). You can use the following command to generate a random bootstrap dataset (“i" is the variable of the loop, so the random_state changes in each iteration): resample(X_train, n_samples = bootstarp_size , random_state=i , replace = True)
Define and train a new base decision tree classifier on this dataset in each iteration: Base_DecisionTree = DecisionTreeClassifier(random_state=2).
Test “this base classifier” on the original “Testing” Dataset build in part(b), and save the prediction results for all testing samples.
Perform Voting to make the final decision on each data sample based on the votes of all 19 classifiers. Finally, calculate and report the accuracy of your Bagging method..</b>

In [26]:
from sklearn.utils import resample
sample_size_data = int(0.8*(len(cancer_data_df)))

list_array = []
i = 0
flag = True

while(flag):
    _x_train = resample(X_train, n_samples = sample_size_data, random_state=i, replace=True)
    _y_train = resample(y_train, n_samples = sample_size_data, random_state=i, replace=True)
    Base_DecisionTree = DecisionTreeClassifier(random_state=2)
    Base_DecisionTree.fit(_x_train, _y_train)
    bag_predict = Base_DecisionTree.predict(X_test)
    list_array.append(bag_predict)
    i = i+1
    if i is 19:
        flag = False


In [27]:
num = -1
prediction_df = pd.DataFrame()
listOfTest = []
for item in (list_array):
    num = num+1
    prediction_df['classifiers_'+ str(num)] = item
    listOfTest.append(item)
print(prediction_df.shape)
prediction_df.tail()

(45, 19)


,classifiers_0,classifiers_1,classifiers_2,classifiers_3,classifiers_4,classifiers_5,classifiers_6,classifiers_7,classifiers_8,classifiers_9,classifiers_10,classifiers_11,classifiers_12,classifiers_13,classifiers_14,classifiers_15,classifiers_16,classifiers_17,classifiers_18
40,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
41,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
42,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
43,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [28]:
####VOTING

vote_templist =  np.zeros((len(prediction_df),), dtype=int)
for i in range(len(prediction_df)):
    voting_zeroes = 0
    voting_ones = 0
    for j in range(len(prediction_df.columns)):
        if listOfTest[j][i] == 1:
            voting_ones = voting_ones + 1
        else:
            voting_zeroes = voting_zeroes + 1
    if voting_zeroes > voting_ones:
        vote_templist[i] = 0
    else:
        vote_templist[i] = 1

bag_accuracy = accuracy_score(y_test, vote_templist)
print("Bagging Accuracy: ",bag_accuracy)


Bagging Accuracy:  0.822222222222


<b>e- Use scikit-learn “Adaboost” classifier to predict Cancer based on the training/testing
datasets that you built in part (b). Then, calculate and report the accuracy of your
classifier. Use this command to import and define your classifier:
from sklearn.ensemble import AdaBoostClassifier
my_AdaBoost = AdaBoostClassifier(n_estimators = 19,random_state=2)</b>

In [33]:
from sklearn.ensemble import AdaBoostClassifier

my_AdaBoost = AdaBoostClassifier(n_estimators = 19,random_state=2)
my_AdaBoost.fit(X_train, y_train)
my_AdaBoost_y_predict = my_AdaBoost.predict(X_test)
my_AdaBoost_accuracy = accuracy_score(y_test, my_AdaBoost_y_predict)
print('AdaBoost:', my_AdaBoost_accuracy)

my_AdaBoost_list = cross_val_score(my_AdaBoost, X, y, cv=10, scoring='accuracy')
my_AdaBoost_cv = my_AdaBoost_list.mean()
print('AdaBoost accuracy with cross validation:',my_AdaBoost_cv)


AdaBoost: 0.933333333333
AdaBoost accuracy with cross validation: 0.929464285714


<b>f- Use scikit-learn “Random Forest” classifier to predict Cancer based on the training/testing
datasets that you built in part (b). Then, calculate and report the accuracy of your
classifier. Use this command to import and define your classifier:
from sklearn.ensemble import RandomForestClassifier
my_RandomForest =
RandomForestClassifier(n_estimators = 19, bootstrap = True, random_state=2)</b>

In [30]:
from sklearn.ensemble import RandomForestClassifier 
my_RandomForest = RandomForestClassifier(n_estimators = 19, bootstrap = True, random_state=2)

In [32]:
my_RandomForest.fit(X_train, y_train)
y_predict = my_RandomForest.predict(X_test)
my_RandomForestAccuracy = accuracy_score(y_test, y_predict)
print("\nAccuracy Random Forest: ", my_RandomForestAccuracy)

randomForest_list = cross_val_score(my_AdaBoost, X, y, cv=10, scoring='accuracy')
my_RandomForest_cv = randomForest_list.mean()
print('RandomForest accuracy with cross validation:',my_RandomForest_cv)



Accuracy Random Forest:  0.955555555556
RandomForest accuracy with cross validation: 0.929464285714
